# Visualisation with Matplotlib
By the end of this section you will learn how to:
- work with Matplotlib directly from Polars
- create bar, row, grouped bar and  scatter charts with Matplotlib
- create time series charts with Matplotlib

See the Seaborn lecture that follows this one for a simpler interface to Matplotlib

In [ ]:
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
csvFile = '../data/titanic.csv'

In [ ]:
df = pl.read_csv(csvFile)
df.head(3)

## Bar chart
We can make a bar chart with Matplotlib by specifying the dataframe columns with the `x` and `height` arguments

In [ ]:
classCounts = (
    df['Pclass']
    .value_counts()
    .sort("Pclass")
    .with_columns(
        pl.col("Pclass").cast(pl.Utf8)
    )
)

classCounts

In [ ]:
classCounts = (
    df['Pclass']
    .value_counts()
    .sort("Pclass")
    .with_columns(
        pl.col("Pclass").cast(pl.Utf8)
    )
)
plt.bar(
    x=classCounts["Pclass"],
    height=classCounts["counts"]
)

Note that we have to cast the `Pclass` column from `Int64` to `Utf8` (string) so that Matplotlib knows it is an ordinal variable rather than a numeric variable.

### Adding title and axis labels
We can format the chart in the normal way for Matplotlib. In this example we add create a `fig` and `ax` object for a chart with a single subplot and add a title and axes labels

In [ ]:
fig, ax = plt.subplots()
ax.bar(
    x=classCounts["Pclass"],
    height=classCounts["counts"],
)
ax.title.set_text("Number of passengers per class")
ax.set_xlabel("Passenger class")
ax.set_ylabel("Number of passengers")
plt.show()

## Row chart
We plot the same data as a horizontal row chart by:
- using `plt.barh` and
- switching the `y` and `width` arguments

In [ ]:
plt.barh(
    y=classCounts["Pclass"],
    width=classCounts["counts"],
)

## Grouped bar chart
We use the example of the survival rate broken down by passenger class from the lecture on group operations using `over`

In [ ]:
survivedPercentageDf = (
    pl.read_csv(csvFile)
    .groupby(["Pclass","Survived"])
    .agg(
        pl.col("Name").count().alias("counts")
    )
    .with_columns(
        (100*(pl.col("counts")/pl.col("counts").sum().over("Pclass"))).round(3).alias("% Survived")
    )
    .sort(["Pclass","Survived"])
)

We create a grouped bar chart by passing the column to group the bars by to the `color` argument

In [ ]:
labels = ['1','2','3']
import numpy as np
x = np.arange(len(labels))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(
    x - width/2, 
    survivedPercentageDf.filter(pl.col("Survived")==0)["% Survived"], 
    width, 
    label='Died'
)
rects2 = ax.bar(
    x + width/2, 
    survivedPercentageDf.filter(pl.col("Survived")==1)["% Survived"], 
    width, 
    label='Survived'
)

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Scores')
ax.set_title('% Survival by class')
ax.set_xticks(x, labels)
ax.legend()

ax.bar_label(rects1, padding=3)
ax.bar_label(rects2, padding=3)

fig.tight_layout()

plt.show()

## Scatter plot and plot size
We make a scatter plot of `log10(Age)` and `log10(Fare`).

We colour the points by survival. To add different colour scatter we must:
- define a list of colours
- loop through the different groups (i.e. survived or not survived)
- add the scatter points for each group onto the `ax` object

We loop through the different groups with the `groupby` function

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))
colours = ['tab:blue', 'tab:orange']
# Loop through the group keys and group DataFrames with groupby
for idx,(survived,survivedDf) in enumerate(
    df.groupby("Survived")
):
    color = colours[idx]
    ax.scatter(
        x=survivedDf["Age"].log(10),
        y=survivedDf["Fare"].log(10),
        c=color, 
        label=survived,
    )
ax.legend()
ax.grid(True)

plt.show()

To control the shape of the chart in Jupyter we set the width and height as a tuple in `plt.subplots` (see above example).

## Line chart
For the line chart we bring in some new data: a time series of ocean wave heights from the buoys near the stormy coast of Ireland

In [ ]:
waveCsvFile = "../data/wave_data.csv"

We are primarily interested in looking at time series of the `significant_wave_height` column. This measures the wave height in metres at each station (i.e. at each buoy).

We show the first two rows of this `DataFrame` here

In [ ]:
(
    pl.read_csv(waveCsvFile)
    .with_columns(
        pl.col("time").str.strptime(pl.Datetime,"%Y-%m-%dT%TZ")
    )
    .select(["time","stationID","significant_wave_height"])
    .head(2)
)

We use `groupby_dynamic` to group the wave data by station and in 1 hour blocks. We take the hourly mean by station.

We apply a filter at the end of this query to restrict the plot to the 6 stations with the largest waves

In [ ]:
stationAggs = (
    pl.read_csv(waveCsvFile)
    .with_columns(
        pl.col("time").str.strptime(pl.Datetime,"%Y-%m-%dT%TZ")
    )
    .groupby_dynamic('time',every="1h",by="stationID")
    .agg(
            pl.col(pl.Float64).mean().suffix("_mean"),
    )
    # Apply a filter to output the stations with the largest waves
    .filter(
        pl.col('significant_wave_height_mean').mean().over("stationID").rank(method='dense',descending=True) < 7
    )
)
stationAggs.head(3)

We can now produce a time series plot of this hourly data by station. To do this we:
- create a `fig` and `ax` object
- loop through `stationAggs` in groups by `stationID`
- add a line to the plot for each station in the loop

In [ ]:
fig,ax = plt.subplots(figsize=(12, 4), dpi=80)
# Loop through the groups to get the DataFrame for each station
for stationID,stationDf in stationAggs.groupby("stationID"):
    # Add a line for each station
    ax.plot(
        stationDf["time"],
        stationDf["significant_wave_height_mean"],
        label=stationID
)
plt.legend()

The chart shows the arrival of some large waves on 25th September 2022

## Exercises
In the exercises you will develop your understanding of:
- creating a range of charts with Matplotlib
- creating charts with control over axis labels and sizing

### Exercise 1
Make a bar chart of the `SibSp` column showing how many passengers there are with 0,1,2 etc siblings. Ensure the chart is correctly ordered in the number of siblings (that means the x-axis must be in order from 0,1,2 etc)

Expand the following cell if you want some hints

In [ ]:
#Hint 1: Do a value counts on the `SibSp` column
#Hint 2: Cast the `SibSp` column to string

In [ ]:
df = pl.read_csv(csvFile)

### Exercise 2
Inspect the columns in the stationsAggs `DataFrame`. Note the `mean_wave_period_mean` column that has the mean wave period (the time between wave crests) in seconds.

Add the `with_columns` statement noted above if the `time` column is not automatically recognised as a datetime

In [ ]:
stationAggs = (
    pl.read_csv(waveCsvFile)
    .with_columns(
        pl.col("time").str.strptime(pl.Datetime,"%Y-%m-%dT%TZ")
    )
    .groupby_dynamic('time',every="1h",by="stationID")
    .agg(
            pl.col(pl.Float64).mean().suffix("_mean"),
    )
    # Apply a filter to output the stations with the largest waves
    .filter(
        pl.col('significant_wave_height_mean').mean().over("stationID").rank(method='dense',descending=True) < 7
    )
)
stationAggs.head(3)

Make a time series plot of the mean wave period showing it has a similar pattern to the wave height plot above

Now make a scatter plot with:
- the significant wave height on the x-axis
- the mean wave period on the y-axis
- coloured by station

Add labels and a title:
- on the x-axis "Wave height (m)"
- on the y-axis "Wave period (s)"
- on the color axis "Station ID"
- for the title "Wave height versus period"

Make the plot area have equal width and height

## Solutions
### Solution to exercise 1
Make a bar chart of the `SibSp` column showing how many passengers there are with 0,1,2 etc siblings. Ensure the chart is correctly ordered in the number of siblings (that means the x-axis must be in order from 0,1,2 etc)

Expand the following cell if you want some hints

In [ ]:
#Hint 1: Do a value counts on the `SibSp` column
#Hint 2: Cast the `SibSp` column to string

In [ ]:
siblingCount = (
    df["SibSp"]
    .value_counts()
    .with_columns(
        pl.col("SibSp").cast(pl.Utf8)
    )
    .sort("SibSp")
)
(
    plt.bar(
        x=siblingCount["SibSp"],
        height=siblingCount["counts"]
    )
)        

### Solution to Exercise 2

Inspect the columns in the stationsAggs `DataFrame`. Note the `mean_wave_period_mean` column that has the mean wave period (the time between wave crests) in seconds.

Make a time series plot of the mean wave period. Ensure the legend is displayed

In [ ]:
fig,ax = plt.subplots(figsize=(12, 4), dpi=80)
# Loop through the groupby to get the values for each station
for idx,(stationID,stationDf) in enumerate(stationAggs.groupby("stationID")):
    # Add a line for each station
    ax.plot(
        stationDf["time"],
        stationDf["mean_wave_period_mean"],
        label=stationID
)
plt.legend()

Make a scatter plot with:
- the significant wave height on the x-axis
- the mean wave period on the y-axis
- coloured by station (with a legend)

In [ ]:
fig, ax = plt.subplots()
colours = ['tab:blue', 'tab:orange','tab:red','tab:green','tab:purple','tab:cyan']
for idx,(stationID,stationDf) in enumerate(stationAggs.groupby("stationID")):
    color = colours[idx]    
    ax.scatter(
        x=stationDf["significant_wave_height_mean"],
        y=stationDf["mean_wave_period_mean"],
        c=color, 
        label=stationID,
        edgecolors='none'
    )
ax.legend()
plt.show()

Add labels and a title:
- on the x-axis "Wave height (m)"
- on the y-axis "Wave period (s)"
- on the color axis "Station ID"
- for the title "Wave height versus period"

In [ ]:
fig, ax = plt.subplots()
colours = ['tab:blue', 'tab:orange','tab:red','tab:green','tab:purple','tab:cyan']
for idx,(stationID,stationDf) in enumerate(stationAggs.groupby("stationID")):
    color = colours[idx]
    
    ax.scatter(
        x=stationDf["significant_wave_height_mean"],
        y=stationDf["mean_wave_period_mean"],
        c=color, 
        label=stationID,
    )
ax.legend()
ax.set_xlabel("Wave height (m)")
ax.set_ylabel("Wave period (s)")
ax.title.set_text("Wave height versus period")

Ensure the plot area has equal width and height

In [ ]:
fig, ax = plt.subplots(figsize=(7,6))
colours = ['tab:blue', 'tab:orange','tab:red','tab:green','tab:purple','tab:cyan']
for idx,(stationID,stationDf) in enumerate(stationAggs.groupby("stationID")):
    color = colours[idx]
    
    ax.scatter(
        x=stationDf["significant_wave_height_mean"],
        y=stationDf["mean_wave_period_mean"],
        c=color, 
        label=stationID,
    )
ax.legend()
ax.set_xlabel("Wave height (m)")
ax.set_ylabel("Wave period (s)")
ax.title.set_text("Wave height versus period")